In [2]:
%load_ext autoreload
%autoreload 1
import sys
import os
from pathlib import Path
sys.path.insert(1, str(Path(os.getcwd()).resolve().parent))

import pandas as pd 
pd.set_option('display.max_columns', 150)

from project.utils.data import *
%aimport project.utils.data

from sktime.forecasting.all import temporal_train_test_split

In [2]:
df = load_extended_data()

In [3]:
### to do

# choose df columns s. t. feature selection results 

In [4]:
df = df.drop(columns=['January', 'February', 'March',
       'April', 'May', 'June', 'July', 'August', 'September', 'October',
       'November', 'December',])

In [5]:
# creating lag features
X = (
    project.utils.data.create_lag_features(df, 'balance', [1, 7, 14])
    .drop(columns=['balance', 'income', 'outcome'])
)

# cutting nulls after creating lags 
start = X[X.isnull().sum(axis=1) == 0].index.min()

y, X = df['balance'][start:], X[start:]

assert y.shape[0] == X.shape[0], 'alarm (shape)'
assert y.index.min() == X.index.min(), 'alarm (dates)'

In [29]:
import numpy as np
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_absolute_error

def time_series_model_fit(target,
                          features,
                          params_grid,
                          model_class,
                          cv_window='expanding',
                          n_splits=5,
                          additional_metric=None,
                          test_size=0.2
                         ):
    # Step 1: Split time series data into train and test
    split_index = int(len(target) * (1 - test_size))
    target_train, target_test = target[:split_index], target[split_index:]
    features_train, features_test = features[:split_index], features[split_index:]

    # Step 2: Define Time Series Cross-Validation
    if cv_window == 'expanding':
        cv = TimeSeriesSplit(n_splits=n_splits if n_splits else len(target_train))
    elif cv_window == 'rolling':
        cv = TimeSeriesSplit(n_splits=n_splits)

    # Step 3: Run Grid Search CV on the train data
    grid_search = GridSearchCV(model_class, params_grid, cv=cv, scoring='neg_mean_absolute_error')
    grid_search.fit(features_train, target_train)

    best_model = grid_search.best_estimator_

    # Step 4: Evaluate the model on the test data
    predictions_test = best_model.predict(features_test)
    mae_test = mean_absolute_error(target_test, predictions_test)

    # Step 5: Fit the best model on all data
    best_model.fit(features, target)

    additional_metric_result = None
    if additional_metric:
        additional_metric_result = additional_metric(target_test, predictions_test)

    return best_model, mae_test, additional_metric_result, grid_search.best_params_



In [30]:
from catboost import  CatBoostRegressor

In [35]:
param_grid = {  
    'iterations': [100, 200, 300],  
    'learning_rate': [0.1, 1],  
    'depth': [5, 7, 8],  
    # 'l2_leaf_reg': [1, 5, 10],  
    # 'random_strength': [1, 5, 10],  
    # 'bagging_temperature': [0.5, 1, 2],  
    # 'random_seed': [0, 42, 123]  
}  
  
model = CatBoostRegressor(verbose=0) 

In [14]:
# y.to_numpy()

In [36]:
result = time_series_model_fit(
    target=y,
    features=X, params_grid=param_grid, model_class=model, cv_window='rolling'
)

In [52]:
y_pred = result[0].predict(X_test)

In [51]:
from sktime.forecasting.all import temporal_train_test_split

y_train, y_test, X_train, X_test = temporal_train_test_split(y, X, test_size=0.1)
cbr_rate_train, cbr_rate_test = df.key_rate[y_train.index], df.key_rate[y_test.index]

In [55]:
# cbr_rate_test

In [57]:
i = 0
project.utils.metrics.calculate_daily_profit(
    prediction=y_pred[i],target=y_test[i],cbr_key_rate=cbr_rate_test[i],
)

/var/folders/yh/4jky_nln065c8nrzc6phgjjdt0k6bs/T/ipykernel_13695/17752452.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  prediction=y_pred[i],target=y_test[i],cbr_key_rate=cbr_rate_test[i],


-0.012365690653529906

In [1]:
i = 0
project.utils.metrics.calculate_add_margin(
    prediction=y_pred[i],target=y_test[i],cbr_key_rate=cbr_rate_test[i],
)

NameError: name 'project' is not defined

In [58]:
project.utils.metrics.calculate_total_earnings(y_pred, y_test, cbr_rate_test)

/Users/ruattar/TS_project/project/utils/metrics.py:40: RuntimeWarning: divide by zero encountered in scalar divide
  profit = (earnings - target) / target
/Users/ruattar/TS_project/project/utils/metrics.py:56: RuntimeWarning: invalid value encountered in scalar multiply
  target * calculate_daily_profit(


nan

In [ ]:
class TS_Model:
    def __init__(self, y, X, test_size=0.1, cv_window, model_class=CatBoostRegressor):
        self.y = y
        self.X = X
        self.test_size=0.1
        self.cv_window = cv_window
        self.

In [39]:
%aimport project.utils.metrics

In [40]:
project.utils.metrics.check_business_requirements

<function project.utils.metrics.check_business_requirements(predictions: pandas.core.series.Series | numpy.ndarray, targets: pandas.core.series.Series | numpy.ndarray, max_deviation: float = 0.42) -> bool>

In [38]:
# class DoubleModel:
#     def __init__(self, income: dict, outcome: dict,  base_model_class,
#                  best_income_model=None, best_outome_model=None,
#                  test_size=0.1
#                 ):
#         self.income = income 
#         self.outcome = outcome
#         self.X = X
#         self.base_model_class = base_model_class
        
#         self.best_income_model = best_income_model
#         self.best_outome_model = best_outome_model

        

#     def fit_series(self, y, X):

        
        